In [1]:
import spacy, os
nlp = spacy.load('en')

In [2]:
# switch for debug
flag_print = True

# switch to clear existing data
flag_clear = True

#threshold value for determining section
threshold = 0.5

In [6]:
# to get extract sections from the resume -- add or remove from  'similar_to' accordingly
similar_to = {
    'edu' : ['education', 'study', 'academics'],
    'exp' : ['job', 'internship', 'training', 'research', 'carrer', 'profession', 
             'project', 'responsibility', 'description'],
    'skill' : ['skill', 'languages', 'technology', 'frameworks', 'tools', 'operating', 'system'],
    'extra' : ['introduction', 'intro', 'achievement', 'hobby', 'links', 'additional', 'personal']
}

list_of_sections = similar_to.keys()

# to bring similar_words to their normal forms
for section in list_of_sections:
    new_list = []
    
    for word in similar_to[section]:
        docx = nlp(unicode(word))
        new_list.append(docx[0].lemma_)
        
    if flag_print:
        print section, new_list
        
    similar_to[section] = new_list

skill [u'skill', u'language', u'technology', u'framework', u'tool', u'operate', u'system']
extra [u'introduction', u'intro', u'achievement', u'hobby', u'link', u'additional', u'personal']
exp [u'job', u'internship', u'training', u'research', u'carrer', u'profession', u'project', u'responsibility', u'description']
edu [u'education', u'study', u'academic']


In [4]:
# function to return the words in a uniform 
def modify(word):
    try:
        symbols = '''~'`!@#$%^&*)(_+-=}{][|\:;",./<>?'''
        mod_word = ''
        
        for char in word:
            if (char not in symbols):
                mod_word += char.lower()

        docx = nlp(unicode(mod_word))

        if (len(mod_word) == 0 or docx[0].is_stop):
            return None
        else:
            return docx[0].lemma_
    except:
        return None # to handle the odd case of characters like 'x02', etc.
    
if flag_print:
    test_words = ['Hello!!', '.,<>', 'India', 'of', '..freedoM..', 'e-mail']
    
    for word in test_words:
        print word, '--returned-->', modify(word)

Hello!! --returned--> hello
.,<> --returned--> None
India --returned--> india
of --returned--> None
..freedoM.. --returned--> freedom
e-mail --returned--> email


In [11]:
if flag_clear:
    for file_name in os.listdir(os.getcwd()+'/prc_data'):
        os.remove(os.getcwd()+'/prc_data/'+file_name)

for file_name in os.listdir(os.getcwd()+'/raw_data'):
    if flag_print:
        print '\n'
        print '*'*25
        print file_name
        print '*'*25
        
    main_file_handler = open('raw_data/'+file_name, 'r')    
    previous_section  = 'extra'
    
    for line in main_file_handler:
        # skip line if empty
        if (len(line.strip()) == 0):
            continue
        
        if flag_print:
            print previous_section, '\t**', line.strip()
        
        # processing next line
        list_of_words_in_line = line.split()
        list_of_imp_words_in_line  = []
        
        for i in range(len(list_of_words_in_line)):
            modified_word = modify(list_of_words_in_line[i])
            
            if (modified_word):
                list_of_imp_words_in_line.append(modified_word)

        if (1 <= len(list_of_imp_words_in_line)):
            curr_line = ' '.join(list_of_imp_words_in_line)
            doc = nlp(unicode(curr_line))
            section_value = {}
            
            # initializing section values to zero
            for section in list_of_sections:
                section_value[section] = 0.0
                
            # updating section values    
            for token in doc:
                for section in list_of_sections:
                    for word in similar_to[section]:
                        word_token = doc.vocab[unicode(word)]
                        section_value[section] = max(section_value[section], float(word_token.similarity(token)))

            # determining the next section based on section values and threshold
            most_likely_section = 'extra'
            for section in list_of_sections:
                #print '>>', section, section_value[section]
                if (section_value[most_likely_section] < section_value[section] and section_value[section] > threshold):
                    most_likely_section = section
            
            # updating the section
            if (previous_section != most_likely_section):
                previous_section = most_likely_section

        # writing data to the separate files
        temp_file_handler = open('prc_data/'+file_name[:-4]+'_'+previous_section+'.txt', 'a')
        temp_file_handler.write(line)
        temp_file_handler.close()
        
    main_file_handler.close()
        



*************************
cv2.txt
*************************
extra 	** Puneet Singh
extra 	** New Delhi, Delhi
extra 	** To work as an IT professional in an IT industry with a professional work driven environment where I can utilize and apply my knowledge, skills and abilities which will beneficial for mutual growth.
skill 	** Personal traits
extra 	** Professional Qualification
exp 	** ➢ MCA […] from BIST, Bhopal affiliated to RGTU with aggregate CGPA 8.48
extra 	** ➢ BCA […] from Integral University Lucknow with aggregate percentage 64.67
edu 	** Work Experience
exp 	** Software Developer
skill 	** Software - Noida, Uttar Pradesh
skill 	** Fresher
extra 	** IT Skills
skill 	** • Technologies -
skill 	** • C - Pointer, Structure, Union, Recursion
skill 	** • C++ - Concepts of oops, virtual method, Abstract method
skill 	** • .Net Technology - C#, Asp.net
skill 	** • HTML
extra 	** • Operating systems - Windows (XP/7/8)
skill 	** • Tools - MS SQL, SQL server, Microsoft Access
skill 	*

exp 	** .
exp 	** Responsibilities:
exp 	** ❖ Responsible for Developing Application Front end using C#
exp 	** ❖ Involved in Designing, Coding and Testing of two modules.
skill 	** ❖ Developed two modules and Visual Studio Crystal Report.
exp 	** ❖ Implemented Database using SQL Server as backend and ADO.Net for their Interaction.
skill 	** ❖ Involved in removing bugs.
exp 	** .
exp 	** Achievements, Extra Curricular Activities:
extra 	** ➢ Active member of Sports meet at college level.
edu 	** ➢ Participation in writing competition in school and college levels.
edu 	** Team Member
extra 	** Appin Software Group
skill 	** September 2010 to May 2011
extra 	** Team Size: 4
extra 	** Technologies: C#, ADO.Net, Crystal Report, SQL Server 2005, VS 2005.
skill 	** App Server: Windows Application
skill 	** Description:
exp 	** Call Data Record Analyzer is used to analysis and tracking criminal on call data record base by most of the intelligence organization of India. This software maintains